In [ ]:
#lonboard warns you about changing the projection upon plotting, I'm filtering this warning
import warnings
warnings.filterwarnings('ignore')

## Playing with a geospatial database 

**Shoutout**: Dr. Wu - Talk at Scipy Friday 10.45am - Room 316


## NYC data database

You can download the data from: https://github.com/opengeos/data/tree/main/duckdb/nyc_data.db.zip
the datasets in the database are in NAD83 / UTM zone 18N projection, EPSG:26918. (https://epsg.io/26918)

To follow this notebook, unzip the file, and put the `nyc_data.db` file in at the same leve of this notebook in your repo.

In [ ]:
import ibis
from ibis import _

ibis.options.interactive = True

con = ibis.duckdb.connect("nyc_data.db")
con.list_tables()

### Problem statement.
Pick a subway station, check in which neighborhood and street is, and then check if there are any homicides that happened within a radius of that station. 

In [ ]:
### this is to avoid a progress bar showing up
ibis.get_backend().raw_sql("PRAGMA disable_progress_bar;");

## Subway stations 

In [ ]:
subway_stations = con.table("nyc_subway_stations")
subway_stations

In [ ]:
broad_station = subway_stations.filter(subway_stations.NAME == "Broad St")
broad_station

In [ ]:
broad_station.geom

In [ ]:
broad_station_coords = broad_station.geom.as_scalar()

In [ ]:
broad_station_coords

## Boroughs and streets tables

In [ ]:
boroughs = con.table("nyc_neighborhoods")
boroughs

### In which Borough is Broad station

In [ ]:
boroughs.filter(boroughs.geom.intersects(broad_station_coords))

### Streets within 10m of broad station

In [ ]:
streets = con.table("nyc_streets")
streets

In [ ]:
sts_near_broad = streets.filter(_.geom.d_within(broad_station_coords, 10))
sts_near_broad

### homicides table

In [ ]:
homicides = con.table("nyc_homicides")
homicides

### Perimeter of 200m around broad station

In [ ]:
broad_station.geom.buffer(200)

In [ ]:
broad_station.geom.buffer(200).area()

### Homicides within 200m of broad station

In [ ]:
h_near_broad = homicides.filter(_.geom.intersects(broad_station.geom.buffer(200).as_scalar()))
h_near_broad

### Where did it happen?

In [ ]:
h_street = streets.filter(_.geom.d_within(h_near_broad.geom.as_scalar(), 2))
h_street

### Let's plot this

**Shoutout**: Kyle Barron - Talk at Scipy Friday 11.25am - Room 316

In [ ]:
from lonboard import Map, ScatterplotLayer, PathLayer, PolygonLayer

**Converting to Geopandas to plot**

In [ ]:
broad_station_gdf = broad_station.to_pandas()
broad_station_gdf.crs = "EPSG:26918"

broad_station_zone = broad_station.mutate(geom=broad_station.geom.buffer(200))
broad_station_zone = broad_station_zone.to_pandas()
broad_station_zone.crs = "EPSG:26918"

streets_gdf = streets.to_pandas()
streets_gdf.crs = "EPSG:26918"

h_near_broad_gdf = h_near_broad.to_pandas()
h_near_broad_gdf.crs = "EPSG:26918"

h_street_gdf = h_street.to_pandas()
h_street_gdf.crs = "EPSG:26918"

**Plotting layers with lonboard**

In [ ]:
broad_station_layer = ScatterplotLayer.from_geopandas(
    broad_station_gdf, get_fill_color="orange", get_radius=5
)

broad_station_zone_layer = PolygonLayer.from_geopandas(
    broad_station_zone, get_fill_color="orange", opacity=0.1
)

h_near_broad_layer = ScatterplotLayer.from_geopandas(
    h_near_broad_gdf, get_fill_color="red", get_radius=5
)

h_street_layer = PathLayer.from_geopandas(
    h_street_gdf, get_color="blue", opacity=0.5, get_width=2
)

streets_layer = PathLayer.from_geopandas(streets_gdf, get_color="grey", opacity=0.3)


mh = Map(
    [
        broad_station_layer,
        broad_station_zone_layer,
        h_near_broad_layer,
        h_street_layer,
        streets_layer,
    ],
    view_state={"longitude": -74.01066, "latitude": 40.7069, "zoom": 16}
)
mh